In [2]:
import os
import sys
import cv2
import shutil
from tqdm import tqdm

In [5]:
SRC_PATH = "D:\Downloads\CCPD2019\CCPD2019"
DES_PATH = "../data"
CLASS_ID = 0

In [ ]:
def parse_file_name(name): 
    parts = name.split('-')
    
    if (len(parts) < 3):
        return None
    
    bounding_box = parts[2]

    try:
        top_left, bottom_right = bounding_box.split('_')
        x_min, y_min = map(int, top_left.split('&'))
        x_max, y_max = map(int, bottom_right.split('&'))
        return x_min, y_min, x_max, y_max
    except:
        return None
    
def read_line(file_path):
    with open(file_path, "r") as f:
        lines = f.read().strip().splitlines()
    return [line.strip() for line in lines if line.strip()]

def YOLO_labels(img_path, x_min, y_min, x_max, y_max):
    img = cv2.imread(img_path)
    h, w = img.shape[:2]

    x_center = ((x_min + x_max) / 2) / w;
    y_center = ((y_min + y_max) / 2) / h;
    width = (x_max - x_min) / w;
    height = (y_max - y_min) / h;

    return f"0 {x_center} {y_center} {width} {height}\n"

def copy_image(files, dataset_name):
    for file in tqdm(files, desc = f"Processing {dataset_name}"):
        src_image = os.path.join(SRC_PATH, file)
        if not os.path.exists(src_image):
            print(f"File {src_image} doesn't exist!\n")
            continue
        file_name = file.split('/')
        file = file_name[1]
        des_image = f"{DES_PATH}/images/{dataset_name}/{file}"
        shutil.copy2(src_image, des_image)

def copy_directory(src_folder, des_folder):
    os.makedirs(des_folder, exist_ok=True)
    files = [f for f in os.listdir(src_folder) if os.path.isfile(os.path.join(src_folder, f))]
    for f in tqdm(files, desc=f"Copying {os.path.basename(src_folder)}"):
        shutil.copy2(os.path.join(src_folder, f), os.path.join(des_folder, f))

In [ ]:
def copy_data(train_files, val_files):
    copy_image(train_files, "train")
    copy_image(val_files, "val")

    copy_directory(f"{SRC_PATH}/ccpd_blur", f"{DES_PATH}/images/test/blur")
    copy_directory(f"{SRC_PATH}/ccpd_tilt", f"{DES_PATH}/images/test/tilt")
    copy_directory(f"{SRC_PATH}/ccpd_rotate", f"{DES_PATH}/images/test/rotate")
    copy_directory(f"{SRC_PATH}/ccpd_fn", f"{DES_PATH}/images/test/fn")
    copy_directory(f"{SRC_PATH}/ccpd_db", f"{DES_PATH}/images/test/db")
    copy_directory(f"{SRC_PATH}/ccpd_challenge", f"{DES_PATH}/images/test/challenge")
    copy_directory(f"{SRC_PATH}/ccpd_weather", f"{DES_PATH}/images/test/weather")

def generate_YOLO_labels():
    image_root = f"{DES_PATH}/images"
    label_root = f"{DES_PATH}/YOLO_labels"
    splits = ["train", "val"]
    for split in splits:
        image_folder = os.path.join(image_root, split)
        label_folder = os.path.join(label_root, split)
        os.makedirs(label_folder, exist_ok=True)
        
        files = [f for f in os.listdir(image_folder) if f.endswith(".jpg")]
        for file in tqdm(files, desc = f"Processing {split}"):
            image_path = os.path.join(image_folder, file)
            x_min, y_min, x_max, y_max = parse_file_name(file)
            label = YOLO_labels(image_path, x_min, y_min, x_max, y_max)
            label_name = file.replace("jpg", "txt")
            label_path = os.path.join(label_folder, label_name)
            with open(label_path, "w") as f:
                f.write(label)
    
    testImage_root = f"{DES_PATH}/images/test"
    testLabel_root = f"{DES_PATH}/YOLO_labels/test"
    test_split = ["blur", "tilt", "rotate", "fn", "db", "challenge", "weather"]
    for split in test_split:
        image_folder = os.path.join(testImage_root, split)
        label_folder = os.path.join(testLabel_root, split)
        os.makedirs(label_folder, exist_ok=True)

        files = [f for f in os.listdir(image_folder) if f.endswith(".jpg")]
        for file in tqdm(files, desc = f"Processing test/{split}"):
            image_path = os.path.join(image_folder, file)
            x_min, y_min, x_max, y_max = parse_file_name(file)
            label = YOLO_labels(image_path, x_min, y_min, x_max, y_max)
            label_name = file.replace("jpg", "txt")
            label_path = os.path.join(label_folder, label_name)
            with open(label_path, "w") as f:
                f.write(label)

In [ ]:
if not os.path.exists(SRC_PATH):
    print("The CCPD folder hasn't existed!")
    sys.exit()
    
if os.path.exists(DES_PATH):
    shutil.rmtree(DES_PATH)
    
os.makedirs(f"{DES_PATH}/images/train", exist_ok = True)
os.makedirs(f"{DES_PATH}/images/val", exist_ok = True)
os.makedirs(f"{DES_PATH}/images/test", exist_ok = True)
os.makedirs(f"{DES_PATH}/YOLO_labels/train", exist_ok = True)
os.makedirs(f"{DES_PATH}/YOLO_labels/val", exist_ok = True)
os.makedirs(f"{DES_PATH}/YOLO_labels/test", exist_ok = True)
os.makedirs(f"{DES_PATH}/FastRCNN_labels/train", exist_ok = True)
os.makedirs(f"{DES_PATH}/FastRCNN_labels/val", exist_ok = True)
os.makedirs(f"{DES_PATH}/FastRCNN_labels/test", exist_ok = True)

train_files = read_line(f"{SRC_PATH}/splits/train.txt")
val_files = read_line(f"{SRC_PATH}/splits/val.txt")
print("Quantity of train:", len(train_files))
print("Quantity of validation:", len(val_files))

In [ ]:
copy_data(train_files, val_files)

In [ ]:
generate_YOLO_labels()

In [3]:
def draw_bounding_box(image_path, top_left, bottom_right, color=(0, 255, 0), thickness=2):
    if not os.path.exists(image_path):
        print(f"Error: Not found image in the path: {image_path}")
        return
    image = cv2.imread(image_path)
    if image is None:
        print("Image can't be read")
        return
    
    output_image = cv2.rectangle(
        img=image,
        pt1=top_left,
        pt2=bottom_right,
        color=color,
        thickness=thickness
    )

    cv2.imshow("Image with Bounding Box", output_image)
    cv2.waitKey(0)
    print("Đã vẽ bounding box và hiển thị ảnh.")

In [4]:
image_path = "../../../data/images/train/01-90_85-274&361_472&420-475&416_277&422_271&357_469&351-0_0_25_29_33_26_30-165-31.jpg"

top_left = (274, 361)
bottom_right = (472, 420)
draw_bounding_box(image_path, top_left, bottom_right)

Đã vẽ bounding box và hiển thị ảnh.
